In [27]:

def get_list(split, path, subsets, category_dict, data_percentage, max_imgs):
    data_list = []
    for subset in subsets:
        for cat in category_dict[subset]:
            list_fname = f"{path}/{subset}/lists/{cat}_{split}.list"
            if not os.path.exists(list_fname):
                continue
            lines = open(list_fname).read().splitlines()
            lines = lines[:round(data_percentage*len(lines))]
            for i, img_fname in enumerate(lines):
                if i >= max_imgs: break
                name = '.'.join(img_fname.split('.')[:-1])
                object_name = name.split('_')[-2]
                sample_id = name.split('_')[-1]
                data_list.append((subset, cat, object_name, sample_id))
    return data_list

def get_pointcloud(path, subset, category, object_name):
    fname = f"{category}/{category}_{object_name}"
    pc_fname = f"{path}/{subset}/pointclouds/{fname}.npy"
    pc = np.load(pc_fname)
    dpc = {"points":torch.from_numpy(pc).float()}
    return dpc
    
def get_camera(subset, category, object_name, sample_id):
    fname = f"{category}/{category}_{object_name}_{sample_id}"
    intr_p = f"{path}/{subset}/camera_data/intr/{fname}.npy"
    extr_p = f"{path}/{subset}/camera_data/extr/{fname}.npy"
    Rt = np.load(extr_p)
    K = torch.from_numpy(np.load(intr_p))
    return K, Rt

def get_gt_sdf(path, subset, category, object_name):
    fname = f"{category}/{category}_{object_name}"
    gt_fname = f"{path}/{subset}/gt_sdf/{fname}.npy"
    gt_dict = np.load(gt_fname, allow_pickle=True).item()
    gt_sample_points = torch.from_numpy(gt_dict['sample_pt']).float()
    gt_sample_sdf = torch.from_numpy(gt_dict['sample_sdf']).float() - 0.003
    return gt_sample_points, gt_sample_sdf

In [29]:
import os
import numpy as np
import torch
import sys

sys.path.append('/home/zubairirshad/ZeroShape')
import utils.camera as camera

path = '/home/zubairirshad/Downloads/train_data'
category_dict = {}
category_list = []

max_imgs = 10
data_percentage = 1

subsets = ['objaverse_LVIS_tiny']
for subset in subsets:
    subset_path = "{}/{}".format(path, subset)
    categories = [name[:-11] for name in os.listdir("{}/lists".format(subset_path)) if name.endswith("_train.list")]
    category_dict[subset] = categories
    category_list += [cat for cat in categories]

lists_all = get_list('train', path, subsets, category_dict, data_percentage, max_imgs)

print("lists_all", lists_all[:10])

list = lists_all[:-10]

# idx = np.random.choice(len(lists_all))
idx = 0
subset, category, object_name, sample_id = lists_all[idx]

print("subset, category, object_name, sample_id", subset, category, object_name, sample_id)

#Get camera pose

# load camera
K, Rt = get_camera(subset, category, object_name, sample_id)
R = np.zeros((3,4))
R[:3,:3] = Rt[:3,:3]
t = Rt[:3,3]
t = camera.pose(t=t)
pose = camera.pose.compose([R, t])

print("K", K)
print("pose", pose)

# load point cloud
dpc = get_pointcloud(path, subset, category, object_name)

# load gt sdf
gt_sample_points, gt_sample_sdf = get_gt_sdf(path, subset, category, object_name)

# print("dpc", dpc.shape)
print("dpc", dpc['points'].shape)
print("gt_sample_points", gt_sample_points.shape)
print("gt_sample_sdf", gt_sample_sdf.shape)



lists_all [('objaverse_LVIS_tiny', 'alarm_clock', '85ad615c7f324095b5116c7ba66489db', '0013'), ('objaverse_LVIS_tiny', 'alarm_clock', '719469569287452f86f1d3568158ec8d', '0001'), ('objaverse_LVIS_tiny', 'alarm_clock', 'aa9b5d16b0be4f12a0cc3acb97b91587', '0020'), ('objaverse_LVIS_tiny', 'alarm_clock', '5192e34141a841149496acfbc41b563f', '0014'), ('objaverse_LVIS_tiny', 'alarm_clock', '4b3adf64384e450f9553ce3fba9882fd', '0023'), ('objaverse_LVIS_tiny', 'alarm_clock', '617edd940bed42159139736f6a20abae', '0020'), ('objaverse_LVIS_tiny', 'alarm_clock', 'e822d0679b6e499a8cb3e9d2d185055b', '0016'), ('objaverse_LVIS_tiny', 'alarm_clock', 'c32c12343e3940d2ad99f702597bcc73', '0025'), ('objaverse_LVIS_tiny', 'alarm_clock', '67a2240611f642068a8896b2df3dfca8', '0018'), ('objaverse_LVIS_tiny', 'alarm_clock', 'b642670e0c3a41b6863762c8545079db', '0017')]
subset, category, object_name, sample_id objaverse_LVIS_tiny alarm_clock 85ad615c7f324095b5116c7ba66489db 0013
K tensor([[592.5926,   0.0000, 107.733